In [1]:
from pydap.client import open_url
import numpy as np
import pandas as pd
from datetime import timedelta
from datetime import date
from tqdm import tqdm

In [2]:
def fetch_oscar_data(start_year, start_month, start_day, end_year, end_month, end_day):
    d0 = date(1992, 10, 5)
    d1 = date(start_year, start_month, start_day)
    d2 = date(end_year, end_month, end_day)
    delta_start = d1 - d0
    delta_end = d2 -d0
    
    for i in tqdm(range(delta_start.days, delta_end.days+1)):
        res = fetch_oscar_one_day(i)
        if res:
            u_list = []
            v_list = []
            lat_list = []
            lon_list = []
            for index, lat in enumerate(res[0]):
                for index2, lon in enumerate(res[1]):
                    u_list.append(res[2][index][index2])
                    v_list.append(res[3][index][index2])
                    lat_list.append(lat)
                    lon_list.append(lon)
            df = pd.DataFrame()
            df['u'] = u_list
            df['v'] = v_list
            df['lat'] = lat_list
            df['lon'] = lon_list
            df['time'] = np.zeros(len(df['lon']))
            df = df.assign(time=i*24)
            df = df.loc[df['lon'] < 380 ]
            df['lon'] = df['lon']%360
            df = df.loc[df['lat'] <= 38 ]
            df = df.loc[df['lat'] >= -38 ]
            df.dropna(inplace = True)
            day = d0 + timedelta(days=i)
            df.to_csv("oskar_data/"  + str(day.year) + "_" +  str(day.month).zfill(2) + "_" +  str(day.day).zfill(2) + ".csv" ,index=False)
        
def fetch_oscar_one_day(day):
    try:
        url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/oscar/preview/L4/oscar_third_deg/oscar_vel' + str(day) + '.nc.gz?latitude[0:1:480],longitude[0:1:1200],u[0:1:0][0:1:0][0:1:480][0:1:1200],v[0:1:0][0:1:0][0:1:480][0:1:1200]'       
        dataset = open_url(url, output_grid=False)
        lats = np.array(dataset.latitude[:])
        lons = np.array(dataset.longitude[:])
        u = np.squeeze(np.array(dataset.u[:][:][:][:]))            
        v = np.squeeze(np.array(dataset.v[:][:][:][:]))         
        return lats, lons, u, v
    except:
        return None

In [3]:
fetch_oscar_data(2017, 3, 15, 2017, 8, 15)

100%|██████████| 154/154 [12:03<00:00,  4.70s/it]


-78.33333333333333